In [157]:
## Required Libraries
import re
import nltk
import string
import pandas as pd
nltk.download('punkt')
nltk.download('twitter_samples')
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.corpus import twitter_samples
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vinu1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\vinu1\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


ModuleNotFoundError: No module named 'textblob'

In [120]:
### Loading the Dataframe cell 

df = pd.read_csv('C:\\Users\\vinu1\\Personality_perception\\data\\Tweets\\BarackObama_tweets.csv')
# df.drop(['Unnamed: 0', 'Index',  
#                 'Location', 'Subcategory',
#        'Unnamed: 5'], axis=1, inplace=True)
# df['text'] = df['text'].str.lower()
# df['text']

In [165]:
class CleanData:
    
    def __init__(self, df):
         self.df_cleaned = df
         self.Openness_list = []
         self.Agreeableness_list = []
         self.all_positive_tweets = []
         self.all_negative_tweets = []
         self.stopwords_english = stopwords.words('english')
        
    def read_persona_dict(self, Personality):
        for i in Personality:
            path = f"C:\\Users\\vinu1\\Personality_perception\\data\\Dictionary\\{i}.txt"
            with open(path) as file:
                if i == "Openness":
                   self.Openness_list = [line.rstrip() for line in file]
                   self.Openness_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Openness_list]
                elif i == "Agreeableness":
                   self.Agreeableness_list = [line.rstrip() for line in file]
                   self.Agreeableness_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Agreeableness_list]

    def pre_processing_dataframe(self):
        self.df_cleaned["text"] = self.df_cleaned["text"].str.lower()
        word_token_list = []
        for i in range(len(self.df_cleaned.index)):
            word_token = word_tokenize(self.df_cleaned["text"][i])
            word_token_list.append(word_token)
        df1 = pd.DataFrame(word_token_list)
        df1['tokenized_tweets']=df1.apply(lambda row: row.dropna().tolist(), axis=1)
        df_token = df1[['tokenized_tweets']]
        self.df_cleaned = pd.concat([self.df_cleaned.reset_index(drop=True), df_token], axis=1)
        
    def get_persona_scores(self):
        max_list = []
        persona_list = []
        open_count, agree_count = 0 , 0
        open_ratio, agree_ratio = 0 , 0

        for i in range(len(self.df_cleaned.index)):
            max_score = 0
            open_count, agree_count = 0 , 0
            open_ratio, agree_ratio = 0 , 0

            for j in range(len(self.df_cleaned["tokenized_tweets"][i])):

                if (self.df_cleaned["tokenized_tweets"][i][j] in self.Openness_list):
                    open_count += 1
                    open_ratio = open_count/len(self.Openness_list)
                if (self.df_cleaned["tokenized_tweets"][i][j] in self.Agreeableness_list):
                    agree_count +=1
                    agree_ratio = agree_count/len(self.Agreeableness_list)

            max_score = max(open_ratio,agree_ratio)
            max_list.append(max_score)

            if max_score == 0:
                x = "Unknown"
            elif max_score == open_ratio:
                x = "Openness"
            elif max_score == agree_ratio:
                x = "Agreeableness"

            persona_list.append(x)


        self.df_cleaned["personality_score"] = max_list
        self.df_cleaned["Personality"] = persona_list
    
    
#     def get_twitter_sample(self):
        
#         punctuations = string.punctuation
#         tokenizer = TweetTokenizer(preserve_case = False, strip_handles=True, reduce_len = True)
#         stemmer = PorterStemmer()

#         self.all_positive_tweets = twitter_samples.strings('positive_tweets.json')
#         self.all_negative_tweets = twitter_samples.strings('negative_tweets.json')
        
        
#         def remove_hyperlinks_marks_styles(tweet):
#             new_tweet = re.sub(r'RT[\s]+','', tweet)
#             new_tweet = re.sub(r"http\S+",'', new_tweet)
#             new_tweet = re.sub(r'#', '',new_tweet)
#             return new_tweet
        
        
#         def tokenize_tweet(tweet):
#             tweet_tokens = tokenizer.tokenize(tweet)
#             return tweet_tokens

        

#         def remove_stopwords_punctuations(tweet_tokens):
#             tweets_clean = []
#             for word in tweet_tokens:
#                if (word not in stopwords_english and word not in punctuations):
#                        tweets_clean.append(word)
#             return tweets_clean
      
#         def get_stem(tweets_clean):
#             tweets_stem = []
#             for word in tweets_clean:
#                stem_word = stemmer.stem(word)
#                tweets_stem.append(stem_word)
#             return tweets_stem
        
    def get_sentiment_score():
        pass

In [159]:
Personality = ['Openness', 'Agreeableness']
clean_data = CleanData(df)
clean_data.read_persona_dict(Personality)
clean_data.pre_processing_dataframe()
clean_data.get_persona_scores()
clean_data.df_cleaned.head()

,id,created_at,text,tokenized_tweets,personality_score,Personality
0,1510286021182771200,2022-04-02 16:00:21+00:00,"i heard betty reid soskin is retiring at 100, ...","[i, heard, betty, reid, soskin, is, retiring, ...",0.000000,Unknown
1,1510050583197237252,2022-04-02 00:24:49+00:00,"i'm proud of titus, ashton, thaddeus, jalen, m...","[i, 'm, proud, of, titus, ,, ashton, ,, thadde...",0.000000,Unknown
2,1507085038940262408,2022-03-24 20:00:48+00:00,today @potus announced that the u.s. will be w...,"[today, @, potus, announced, that, the, u.s., ...",0.016393,Agreeableness
3,1506730193712467970,2022-03-23 20:30:46+00:00,as the first woman to serve as america’s top d...,"[as, the, first, woman, to, serve, as, america...",0.013699,Openness
4,1506718741459091464,2022-03-23 19:45:16+00:00,"the other day, i called up donnamarie, steve, ...","[the, other, day, ,, i, called, up, donnamarie...",0.000000,Unknown


In [160]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
tweet_example = all_positive_tweets[2277]
tweet_example

'My beautiful sunflowers on a sunny Friday morning off :) #sunflowers #favourites #happy #Friday off… https://t.co/3tfYom0N1i'